In [18]:
import numpy as np

# 1. 계산 그래프

여기에서 그래프는 우리가 잘 아는 그래프 자료구조로, 복수의 노드와 에지로 표현한다.

- 노드: Operation
- 에지: Data

# 2. 연쇄법칙

- 덧셈 노드 역전파: 1 * dL / dx
- 곱셈 노드 역전파: y * dL / dx

# 3. 단순한 계층 구현하기

## 1. 곱셈계층

In [5]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
        
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

### Forward

In [9]:
apple = 100
apple_num = 2
tax = 1.1

# layers
mul_appple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_appple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(int(price))

220


### Backward

In [8]:
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_appple_layer.backward(dapple_price)

print(dapple, int(dapple_num), dtax)

2.2 110 200


## 2. 덧셈 계층

In [10]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [15]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_appple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_appple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# backward
dprice = 1

dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_appple_layer.backward(dapple_price)

print(int(price))
print(dapple_num, int(dapple), dorange, int(dorange_num), dtax)

715
110.00000000000001 2 3.3000000000000003 165 650


# 4. 활성화 함수 계층 구현하기

## 1. RELU 계층

In [16]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

In [20]:
x = np.array([[1.0, -0.5],
              [-2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [22]:
mask = (x <= 0)
print(mask)

[[False  True]
 [ True False]]


RELU 계층은 전기 회로의 스위치에 비유할 수 있습니다. 순전파 때 전류가 흐르고 있으면 스위치를 ON으로 하고, 흐르지 않으면 OFF로 합니다. 역전파 때는 스위치가 ON이라면 전류가 그대로 흐르고, OFF면 더 이상 흐르지 않습니다.

## 2. SIGMOID 계층

In [23]:
class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1 (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx

## 3. Affine/Softmax 계층 구현하기

### 1. Affine 계층

In [27]:
X = np.random.randn(2)
W = np.random.randn(2, 3)
B = np.random.randn(3)

print(X.shape)
print(W.shape)
print(B.shape)

Y = np.dot(X, W) + 3
print(Y.shape)

(2,)
(2, 3)
(3,)
(3,)


행렬의 역전파

- dL / dx = dL / dy * W^t
- dL / dw = X^t * dL / dy

### 2. 배치용 Affine 계층

In [28]:
X_dot_W = np.array([[0, 0, 0],
                   [10,10,10]])
B = np.array([1,2,3])

In [29]:
dY = np.array([[1,2,3],[4,5,6]])
dB = np.sum(dY, axis=0)

In [31]:
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [32]:
dB

array([5, 7, 9])

In [34]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx

### 2. Softmax with Loss

In [37]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        
        # loss
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        
        # batch
        dx = (self.y - self.t) / batch_size
        return dx
        
        
        
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c) # explode 방지
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y)) / batch_size

# 5. 신경망 학습의 전체 그림

## 신경망 학습 순서

전체
- 신경망에는 적응 가능한 가중치와 편향이 있고 이 가중치와 편향을 훈련 데이터 적응하도록 조정하는 과정을 학습이라 한다. 신경망 학습은 다음 4단계로 수행한다.

1단계: 미니배치
- 훈련 데이터 중 일부를 무작위로 가져옵니다. 이렇게 선별한 데이터를 미니배치라 하며, 그 미니배치의 손실 함수 값을 줄이는 것이 목표입니다.

2단계: 기울기 산출
- 미니배치 손실 함수 값을 줄이기 위해 각 가중치 매개변수 기울기를 구합니다. 기울기는 손실 함수 값을 가장 작게 하는 방향을 제시합니다.

3단계: 매개변수 갱신
- 가중치 배개변수를 기울기 방향으로 아주 조금 갱신합니다.

4단계: 반복
- 1~3단계를 반복합니다.

In [39]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

In [45]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        
        # initialize params
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # create layers
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()
        
    # forward prediction
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
            
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        
        if t.ndim != 1:
            t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
        
        # forward
        self.loss(x, t)
        
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse() 
        
        # backward
        for layer in layers:
            dout = layer.backward(dout)
            
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

### 오차역전파법으로 구한 기울기 검증하기

In [46]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))

W1:5.01654722559081e-10
b1:3.1392828282758123e-09
W2:5.813795604783152e-09
b2:1.3974995704407744e-07


### 오차역전파를 사용한 학습 구현하기

In [47]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 오차 역전파 법으로 기울기를 구한다
    grad = network.gradient(x_batch, t_batch)
    
    # update
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.13783333333333334 0.1459
0.9034333333333333 0.907
0.92345 0.9285
0.9344 0.9353
0.9436666666666667 0.9405
0.95185 0.9498
0.95525 0.953
0.9599833333333333 0.9562
0.9649333333333333 0.9598
0.967 0.962
0.969 0.9632
0.9717 0.9659
0.9740833333333333 0.968
0.9760166666666666 0.9693
0.9769 0.9709
0.9785333333333334 0.9711
0.9795 0.9697
